In [2]:
import numpy as np
import scipy.sparse as sparse
from scipy.sparse import linalg
import pandas as pd
from scipy.io import loadmat
from scipy.io import savemat
from matplotlib.pyplot import *
import matplotlib.pyplot as plt

In [3]:
# The ESN functions for training, adjacency matric A represent the connectivity of the reservoir neurons
def generate_reservoir(size,radius,degree):
    sparsity = degree/float(size)
    A = sparse.rand(size,size,density=sparsity).todense()
    vals = np.linalg.eigvals(A) 
    e = np.max(np.abs(vals))
    A = (A/e) * radius 
    return A 

In [4]:
def reservoir_layer(A, Win, input, res_params):
    states = np.zeros((res_params['N'],res_params['train_length']))
    for i in range(res_params['train_length']-1):
        states[:,i+1] = np.tanh(np.dot(A,states[:,i]) + np.dot(Win,input[:,i]))
    return states 

In [5]:
def train_reservoir(res_params, data):
    A = generate_reservoir(res_params['N'], res_params['radius'], res_params['degree'])
    q = int(res_params['N']/res_params['num_inputs'])
    Win = np.zeros((res_params['N'],res_params['num_inputs']))
    for i in range(res_params['num_inputs']):
        np.random.seed(seed=i)
        Win[i*q: (i+1)*q,i] = res_params['sigma'] * (-1 + 2 * np.random.rand(1,q)[0])
        
    states = reservoir_layer(A, Win, data, res_params)
    Wout = train(res_params, states, data)
    x = states[:,-1]
    return x, Wout, A, Win

In [6]:
def train(res_params,states,data):
    beta = res_params['beta'] 
    idenmat = beta * sparse.identity(res_params['N'])
    states2 = states.copy()
    for j in range(2,np.shape(states2)[0]-2):
        if (np.mod(j,2)==0):
            states2[j,:] = (states[j-1,:]*states[j-2,:]).copy()
    U = np.dot(states2,states2.transpose()) + idenmat
    Uinv = np.linalg.inv(U)
    Wout = np.dot(Uinv,np.dot(states2,data.transpose()))
    return Wout.transpose()

In [7]:
def predict(A, Win, res_params, x, Wout):
    output = np.zeros((res_params['num_inputs'],res_params['predict_length']))
    for i in range(res_params['predict_length']):
        x_aug = x.copy()
        for j in range(2,np.shape(x_aug)[0]-2):
            if (np.mod(j,2)==0):
                x_aug[j] = (x[j-1]*x[j-2]).copy()
        out = np.squeeze(np.asarray(np.dot(Wout,x_aug)))
        output[:,i] = out
        x1 = np.tanh(np.dot(A,x) + np.dot(Win,out))
        x = np.squeeze(np.asarray(x1))
    return output, x

In [8]:
# compute RMSE
def computeRMSE(X_pred, X_true):
    X_pred_avg = X_pred[:,:,:]
    X_true_avg = X_true[:,:,:]
    err = X_true_avg-X_pred_avg
    err_temp = np.multiply(err, err)
    RMSE_ini=np.sqrt(np.mean(err_temp, axis=0)) # [time_step,ic]
    RMSE_time=np.mean(RMSE_ini,axis=1) # [time_step]
    return RMSE_ini,RMSE_time

In [59]:
# compute ACC
def computeAnomalyCorrelationCoefficient(X_pred, X_true):
    AC_vec = np.zeros((X_pred.shape[1], X_pred.shape[2]))  # X_pred[feature,time_step,ic(initial condition)]
    for ic in range(X_pred.shape[2]):
        X_pred_avg = np.matrix(X_pred[:,:,ic])
        X_true_avg = np.matrix(X_true[:,:,ic])
        X_true_mean = np.mean(X_true[:,:,ic], axis=1)  # real mean in whole time [feature]
        temp1 = np.multiply((X_pred_avg-(np.matrix(X_true_mean)).T),(X_true_avg-(np.matrix(X_true_mean)).T))
        temp1 = np.sum(temp1, axis=0)  # [time_step]
        temp2 = np.multiply((X_pred_avg-(np.matrix(X_true_mean)).T),(X_pred_avg-(np.matrix(X_true_mean)).T))
        temp2 = np.sqrt(np.sum(temp2, axis=0))  # [time_step]
        temp3 = np.multiply((X_true_avg-(np.matrix(X_true_mean)).T),(X_true_avg-(np.matrix(X_true_mean)).T))
        temp3 = np.sqrt(np.sum(temp3, axis=0))  # [time_step]
        AC_vec[:,ic] = temp1/np.multiply(temp2,temp3)  # [time_step,ic]
    print("Mean anomaly correlation coefficient over"+str(X_pred.shape[2])
          + " different initial conditions")
    AC = np.mean(AC_vec, axis=1)  # [time_step] 
    return AC, AC_vec

In [9]:
file_H='1D S-V\1D S-V data.mat'
dataf_H=loadmat(file_H,mat_dtype=True)
dataf_H=dataf_H['Hrecord']
data_H = np.transpose(np.array(dataf_H[15000:,1:202])) # [feature, time_step]

In [24]:
Y_true=np.zeros((res_params['num_inputs'],res_params['predict_length'],28)) # [feature,time_step,ic(initial condition)]
Y_pred=np.zeros((res_params['num_inputs'],res_params['predict_length'],28))

In [29]:
for i in range(28):
    shift_k = 3000*i
    print(i,shift_k)
    # Train reservoir
    x,Wout,A,Win = train_reservoir(res_params,data_H[:,shift_k:shift_k+res_params['train_length']])
    # Prediction
    output, _ = predict(A, Win,res_params,x,Wout)
    Y_true[:,:,i]=data_H[:,shift_k+res_params['train_length']:\
                           shift_k+res_params['train_length']+res_params['predict_length']]
    Y_pred[:,:,i]=output

0 0
1 3000
2 6000
3 9000
4 12000
5 15000
6 18000
7 21000
8 24000
9 27000
10 30000
11 33000
12 36000
13 39000
14 42000
15 45000
16 48000
17 51000
18 54000
19 57000
20 60000
21 63000
22 66000
23 69000
24 72000
25 75000
26 78000
27 81000


In [61]:
RMSE_ini,RMSE_time=computeRMSE(Y_pred, Y_true)
AC, AC_vec=computeAnomalyCorrelationCoefficient(Y_pred, Y_true)
savemat('Res_size'+str(approx_res_size)+'_Rd_'+str(res_params['radius'])+'_Shift_number_28'\
        +'.mat',{'Y_true':Y_true,'Y_pred':Y_pred,'RMSE_ini':RMSE_ini,'RMSE_time':RMSE_time,\
                 'AC':AC,'AC_vec':AC_vec})

Mean anomaly correlation coefficient over28 different initial conditions


# Res_size, IC1

In [17]:
Y_true_Res=np.zeros((200,500,25)) # [feature,time_step,RS(Res_size)]
Y_pred_Res=np.zeros((200,500,25))

In [18]:
for i in range(25):
    approx_res_size = 200*i+200
    shift_k = 3000*0
    print(i,approx_res_size)
    
    model_params = {'N': 200}

    res_params = {'radius':0.1, 
             'degree': 3, 
             'sigma': 0.5,
             'train_length': 2000,
             'N': int(np.floor(approx_res_size/model_params['N']) * model_params['N']), # 储备池维度
             'num_inputs': model_params['N'],
             'predict_length': 500,
             'beta': 0.0001 
              }
    # Train reservoir
    x,Wout,A,Win = train_reservoir(res_params,data_H[:,shift_k:shift_k+res_params['train_length']])
    # Prediction
    output, _ = predict(A, Win,res_params,x,Wout)
    Y_true_Res[:,:,i]=data_H[:,shift_k+res_params['train_length']:\
                           shift_k+res_params['train_length']+res_params['predict_length']]
    Y_pred_Res[:,:,i]=output

0 200
1 400
2 600
3 800
4 1000
5 1200
6 1400
7 1600
8 1800
9 2000
10 2200
11 2400
12 2600
13 2800
14 3000
15 3200
16 3400
17 3600
18 3800
19 4000
20 4200
21 4400
22 4600
23 4800
24 5000


In [19]:
RMSE_ini,RMSE_time=computeRMSE(Y_pred_Res, Y_true_Res)

In [20]:
savemat('Res_size_25'+'_Rd_'+str(res_params['radius'])+'_Shift_number_1'\
        +'.mat',{'Y_true':Y_true_Res,'Y_pred':Y_pred_Res,'RMSE_ini':RMSE_ini,'RMSE_time':RMSE_time})

# Raduis, IC1

In [9]:
Y_true_Rd=np.zeros((200,500,100)) # [feature,time_step,RS(Res_size)]
Y_pred_Rd=np.zeros((200,500,100))

In [ ]:
for i in range(100):
    approx_res_size = 1400
    shift_k = 3000*0
    
    model_params = {'N': 200}

    res_params = {'radius':0.01+i*0.01,
             'degree': 3, # 平均度
             'sigma': 0.5,
             'train_length': 2000,
             'N': int(np.floor(approx_res_size/model_params['N']) * model_params['N']), # 储备池维度
             'num_inputs': model_params['N'],
             'predict_length': 500, 
             'beta': 0.0001 
              }
    print(i,res_params['radius'])
    # Train reservoir
    x,Wout,A,Win = train_reservoir(res_params,data_H[:,shift_k:shift_k+res_params['train_length']])
    # Prediction
    output, _ = predict(A, Win,res_params,x,Wout)
    Y_true_Rd[:,:,i]=data_H[:,shift_k+res_params['train_length']:\
                           shift_k+res_params['train_length']+res_params['predict_length']]
    Y_pred_Rd[:,:,i]=output

In [11]:
RMSE_ini,RMSE_time=computeRMSE(Y_pred_Rd, Y_true_Rd)

In [12]:
savemat('Res_size_1400'+'_Rd_100'+'_Shift_number_1'\
        +'.mat',{'Y_true':Y_true_Rd,'Y_pred':Y_pred_Rd,'RMSE_ini':RMSE_ini,'RMSE_time':RMSE_time})

# trianing_size

In [13]:
Y_true_Rd=np.zeros((200,500,83)) # [feature,time_step,RS(Res_size)]
Y_pred_Rd=np.zeros((200,500,83))

In [14]:
for i in range(83):
    approx_res_size = 1400
    shift_k = 0
    train_length=1000*i+1000
    
    model_params = {'N': 200}

    res_params = {'radius':0.1,
             'degree': 3,
             'sigma': 0.5,
             'train_length': train_length,
             'N': int(np.floor(approx_res_size/model_params['N']) * model_params['N']), # 储备池维度
             'num_inputs': model_params['N'], 
             'predict_length': 500,
             'beta': 0.0001
              }
    print(i,res_params['train_length'])
    # Train reservoir
    x,Wout,A,Win = train_reservoir(res_params,data_H[:,shift_k:shift_k+res_params['train_length']])
    # Prediction
    output, _ = predict(A, Win,res_params,x,Wout)
    Y_true_Rd[:,:,i]=data_H[:,shift_k+res_params['train_length']:\
                           shift_k+res_params['train_length']+res_params['predict_length']]
    Y_pred_Rd[:,:,i]=output

0 1000
1 2000
2 3000
3 4000
4 5000
5 6000
6 7000
7 8000
8 9000
9 10000
10 11000
11 12000
12 13000
13 14000
14 15000
15 16000
16 17000
17 18000
18 19000
19 20000
20 21000
21 22000
22 23000
23 24000
24 25000
25 26000
26 27000
27 28000
28 29000
29 30000
30 31000
31 32000
32 33000
33 34000
34 35000
35 36000
36 37000
37 38000
38 39000
39 40000
40 41000
41 42000
42 43000
43 44000
44 45000
45 46000
46 47000
47 48000
48 49000
49 50000
50 51000
51 52000
52 53000
53 54000
54 55000
55 56000
56 57000
57 58000
58 59000
59 60000
60 61000
61 62000
62 63000
63 64000
64 65000
65 66000
66 67000
67 68000
68 69000
69 70000
70 71000
71 72000
72 73000
73 74000
74 75000
75 76000
76 77000
77 78000
78 79000
79 80000
80 81000
81 82000
82 83000


In [15]:
RMSE_ini,RMSE_time=computeRMSE(Y_pred_Rd, Y_true_Rd)

In [16]:
savemat('train_length_83'+'_Res_size_1400'+'_Rd_0.1'+'_Shift_number_0'\
        +'.mat',{'Y_true':Y_true_Rd,'Y_pred':Y_pred_Rd,'RMSE_ini':RMSE_ini,'RMSE_time':RMSE_time})